In [ ]:
class Course:
  def __init__ (self, course_id= None, name = None, credits = None, capacity = None, prerequisites = None, lecturer = None, schedule_slot = None, enrolled_students = None):
    self.course_id = course_id
    self.name = name
    self.credits = int(credits)
    self.capacity = int(capacity)
    self.prerequisites = prerequisites or []
    self.lecturer = lecturer
    self.schedule_slot = schedule_slot
    self.enrolled_students = enrolled_students or []

  # --עזרים פנימיים--
  def _slots_overlap(self, a, b):
        if a is None or b is None:
            return False
        day_a, start_a, end_a = a
        day_b, start_b, end_b = b
        if day_a != day_b:
            return False
        return max(start_a, start_b) < min(end_a, end_b)

  def is_available(self,student):
    for course in student.current_enrollments:
      if self._slots_overlap(self.schedule_slot, course.schedule_slot):
        return False
    return True

    #--מתודות נדרשות--

  def assign_lecturer(self, lecturer): #--רישום מרצה לקורס--
    if lecturer.is_available(self.schedule_slot):
      if self.lecturer is None:
        self.lecturer = lecturer
        lecturer.assigned_courses.append(self)
      else:
        print("Error: This course already has an assigned lecturer.")
    else:
      print("Error: Not available")


  def set_schedule(self, schedule_slot): #--קביעת מועד לקורס--
    #--חפיפה אצל המרצה--
    if self.lecturer and not self.lecturer.is_available(schedule_slot):
      print("Scheduling conflict in the lecturer's timetable.")
      return

    # חפיפה עם סטודנטים רשומים
    for student in self.enrolled_students:
        for course in student.current_enrollments:
                if course is not self:
                    if self._slots_overlap(schedule_slot, course.schedule_slot):
                        print(f"Scheduling conflict in {student.name}'s timetable.")
                        return

    self.schedule_slot = schedule_slot



  def has_capacity(self): #--בדיקת מקום פנוי--
    return self.capacity > len(self.enrolled_students)




  def meets_prerequisites(self, student): #--בדיקת עמידה בדרישום קדם--
    for course in self.prerequisites:
      if course not in [c for c, g in student.completed_courses if g >= 56]:
        return False
    return True



  def enroll(self, student): ##--רישום סטודנט--
    if student in self.enrolled_students: #כבר רשום
      return True
    if not self.has_capacity(): #אין מקום
      return False
    if not self.meets_prerequisites(student): #לא עומד בדרישות קדם
      return False
    if not self.is_available(student): #התנגשות בלוז
      return False
    self.enrolled_students.append(student)
    student.current_enrollments.append(self)
    return True



  def drop(self, student): #--הסרת רישום--
    if student in self.enrolled_students:
      self.enrolled_students.remove(student)
      student.current_enrollments.remove(self)

In [ ]:
class Student:
  def __init__(self, id_student, name): # בנאי למחלקת סטודנט
    self.id_student = id_student
    self.name = name
    self.completed_courses = []   #רשימה ריקה של קורסים שהושלמו
    self.current_enrollments = [] # רשימה ריקה של קורסים נוכחיים

  def  add_completed_course(self,course, grade):# מתודה שמוסיפה קורסים שהושלמו, כולל הציון הסופי
    if grade < 0: #שגיאה במידה והציון מתחת ל-0
            raise ValueError("error: grade cannot be under 0")

    self.completed_courses.append((course, grade)) # הוספת הקורס לרשימה כולל הציון שלו

    if grade < 56: # הערה אם הציון מתחת ל56 (עובר)
            print(f"{self.name} didn't pass the course {course.name} (Grade: {grade}).")
    else:
            print(f"course {course.name} was added with grade {grade}.")


  def calculate_gpa (self): # חישוב הממוצע

    total_points = 0
    total_credits = 0

    for course, grade in self.completed_courses: # לולאה שעוברת על הקורסים שהושלמו וסוכמת את מספר הנק"ז ואת הציון בכל קורס
            total_points += grade * course.credits
            total_credits += course.credits

    if total_credits == 0:
      return 0  # מונע חלוקה ב-0

    return total_points / total_credits # מחשב את הממוצע המשוקלל


  def report_grades(self): # מייצר רשימה של פרטי הקורס והציון הסופי של כל קורסי הסטודנט

    grades_list = []
    for course, grade in self.completed_courses: # יצירת רשימה עם הפרטים הרלוונטים
            grades_list.append({"course_id": course.course_id, "course_name": course.name,"grade": grade,"credits": course.credits})
    return grades_list


  def meeting_request (self, course, time_slot): # מתודה שקובעת פגישה עם מרצה בקורס מסוים
    lecturer = course.lecturer

    if lecturer and lecturer.is_available(time_slot): # בדיקה האם המרצה זמין בחלון זמן זה
      # lecturer.is_available = False #שינוי ללא זמין - This line seems incorrect, availability should be checked not set here
      print(f"Meeting with {lecturer.name} in course {course.name} is schedualed to {time_slot}.")
      return True
    else:
        print(f"{lecturer.name if lecturer else 'Lecturer'} is unavailable on {time_slot}.") #המרצה אינו זמין
        return False


  def enroll_to_course(self, course): #רישום הסטודנט לקורס

      if course in self.current_enrollments: #במידה והקורס כבר נמצא ברשימת הקורסים של הסטודנט
           print(f"{self.name} already assigned to course {course.name}.")
           return False

      # Use the enroll method of the Course class
      return course.enroll(self)

In [ ]:
class Lecturer:
    def __init__(self, lecturer_id, name):
        self.lecturer_id = lecturer_id
        self.name = name
        self.assigned_courses = []
        self.teaching_log = []

    def assign_course(self, course):

        if course not in self.assigned_courses:
            self.assigned_courses.append(course)
            course.lecturer = self
            print(f"The course {course.name} assigned to lecturer {self.name}.")
        else:
            print(f"The course {course.name} already belongs to the lecturer {self.name}.")

    def remove_course(self, course_id):

        for course in self.assigned_courses:
            if course.course_id == course_id:
                self.assigned_courses.remove(course)
                course.lecturer = None
                print(f"The course {course.name} removed from lecturer {self.name}.")
                return
        print(f"The course {course_id} is not assigned to the lecturer {self.name}.")

    def is_available(self, slot=None):

        for course in self.assigned_courses:
            if course.schedule_slot and course.schedule_slot == slot:
                #print(f"The lecturer {self.name} is not available on {slot}.")
                return False
        #print(f"The lecturer {self.name} is available on {slot}.")
        return True

    def submit_final_grade(self, course):

        if course not in self.assigned_courses:
            print(f"You are not assigned to course {course.name}.")
            return

        weights = {"attendance": 0.05, "homework": 0.25, "exam": 0.7}


        if not hasattr(course, "component_grades"):
            course.component_grades = {}

        for student in course.enrolled_students:

            attendance = float(input(f"Enter attendance grade for {student.name}: "))
            homework = float(input(f"Enter homework grade for {student.name}: "))
            exam = float(input(f"Enter exam grade for {student.name}: "))

            final_grade = round(
                attendance * weights["attendance"] +
                homework * weights["homework"] +
                exam * weights["exam"], 2
            )

            #if not hasattr(student, "final_grades"): # This is causing issues when student has final grades for other courses
            #    student.final_grades = {}
            if not hasattr(student, 'completed_courses'):
                student.completed_courses = []
            student.add_completed_course(course, final_grade)


            course.component_grades[student.student_id] = {
                "attendance": attendance,
                "homework": homework,
                "exam": exam
            }

            print(f"{student.name} final grade: {final_grade}")

    def analyze_teaching_statistics(self):

        report = {}

        for course in self.assigned_courses:
            students = course.enrolled_students
            total_students = len(students)
            if total_students == 0:
                continue

            # Calculate average final grade for the course
            total_final_grade = 0
            students_with_grades = 0
            for student in students:
                for completed_course, grade in student.completed_courses:
                    if completed_course == course:
                        total_final_grade += grade
                        students_with_grades += 1
                        break # Assuming only one final grade per student per course

            avg_final = total_final_grade / students_with_grades if students_with_grades > 0 else 0

            avg_attendance = sum(course.component_grades.get(student.student_id, {}).get("attendance", 0) for student in students) / total_students
            avg_homework = sum(course.component_grades.get(student.student_id, {}).get("homework", 0) for student in students) / total_students
            avg_exam = sum(course.component_grades.get(student.student_id, {}).get("exam", 0) for student in students) / total_students


            report[course.name] = {
                "total_students": total_students,
                "avg_final_grade": round(avg_final, 2),
                "avg_attendance": round(avg_attendance, 2),
                "avg_homework": round(avg_homework, 2),
                "avg_exam": round(avg_exam, 2)
            }

        return report

In [ ]:
class Department:
  def __init__(self, name = None, students = None, lecturers = None, courses = None):
    self.name = name
    self.students = students or []
    self.lecturers = lecturers or []
    self.courses = courses or []


  def add_student(self, student): #--הוספת סטודנט למחלקה--
    if student not in self.students:
      self.students.append(student)
    else:
      print("student already in this department")

  def add_lecturer(self, lecturer): #--הוספת מרצה למחלקה--
    if lecturer not in self.lecturers:
      self.lecturers.append(lecturer)
    else:
      print("lecturer already in this department")


  def add_course (self, course): #--הוספת קורס למחלקה--
    if course not in self.courses:
      self.courses.append(course)
    else:
      print("course already in this department")

  def generate_schedule_report(self): #--יוצר לוח שעות--
    schedule = {}
    for course in self.courses:
      slot = course.schedule_slot
      if slot not in schedule:
        schedule[slot] = []
      schedule[slot].append(course.name)
    overlaps = []
    for slot, course_names in schedule.items():
      if len(course_names) > 1:
        overlaps.append((slot, course_names))

    return schedule, overlaps


  def search_courses (self, search_term):
    results = []
    if isinstance(search_term, (int, float)):# אם מילת המפתח הינה מספר
            for course in self.courses:
                if course.credits == search_term or course.capacity == search_term or course.course_id == search_term:
                    results.append(course)
            return results

    else:
      term = str(search_term).lower() # אם מילת המפתח היא מחרוזת

      for course in self.courses: # חיפוש בשם הקורס או במזהה
                   if (term in course.name.lower()):
                      results.append(course)


      for lecturer in self.lecturers:
        if term in lecturer.name.lower(): # מוסיפים את כל הקורסים שהמרצה הזה מלמד
            for course in self.courses:
                if course.lecturer == lecturer and course not in results:
                    results.append(course)

    return results








In [ ]:
class Book:
    def __init__(self, book_id, title, author, course):
        self.book_id = book_id
        self.title = title
        self.author = author
        self.course = course
        self.is_borrowed = False
        self.borrower = None

    def book_info(self):
        status = f"Borrowed by {self.borrower.name}" if self.is_borrowed else "Available"
        print(f"Book ID: {self.book_id}, Title: {self.title}, Author: {self.author}, Course: {self.course.name}, Status: {status}")

    def borrow(self, student):
        if self.is_borrowed:
            print(f"The book {self.title} by {self.author} is already borrowed by {self.borrower.name}.")
        else:
            self.is_borrowed = True
            self.borrower = student
            print(f"{student.name} borrowed the book {self.title} by {self.author}.")

    def return_book(self):
        if not self.is_borrowed:
            print(f"The book {self.title} by {self.author} is not borrowed.")
        else:
            print(f"{self.borrower.name} returned the book {self.title} by {self.author}.")
            self.is_borrowed = False
            self.borrower = None

In [ ]:
class libary:
  def __init__(self, name):
    self.name = name
    self.books = []

  def add_book(self, book): #--הוספת ספר--
    if book not in self.books:
      self.books.append(book)

  def search(self, keyword): #--חיפוש ספר--
    result = []
    for b in self.books:
      if keyword in b.title or keyword in b.author:
        result.append(b)
    return result

  def borrowed_report(self):
    borrowed_books = [book.title for book in self.books if book.is_borrowed]
    return borrowed_books

In [ ]:
dep = Department("הנדסת תעשייה וניהול")

# 2. יצירת מרצים
lec1 = Lecturer("L1", "Dr. Cohen")
lec2 = Lecturer("L2", "Dr. Levy")
dep.add_lecturer(lec1)
dep.add_lecturer(lec2)

# 3. יצירת קורסים
c1 = Course("C101", "מבוא לתעשייה וניהול", 3, 30)
c2 = Course("C102", "סטטיסטיקה", 4, 40, prerequisites=[c1]) # prerequisites should be a list
dep.add_course(c1)
dep.add_course(c2)

# 4. יצירת סטודנטים
s1 = Student("322565895", "Adi")
s2 = Student("S2332454858", "Or")
dep.add_student(s1)
dep.add_student(s2)

# 5. שיוך מרצים לקורסים
c1.assign_lecturer(lec1)
c2.assign_lecturer(lec2)

# 6. קביעת מערכת שעות
c1.set_schedule(("Mon", 10, 12))
c2.set_schedule(("Mon", 10, 12))  # בכוונה חפיפה כדי לראות שהמערכת מזהה

# 7. סימון קורסים שהסטודנט השלים כדי לבדוק דרישות קדם
s2.add_completed_course(c1, 90) # Pass the Course object, not the name

# 8. רישום סטודנטים לקורסים
print("\n--- רישום לקורסים ---")
print("רישום עדי לקורס C101:", c1.enroll(s1))
print("רישום אור לקורס C102:", c2.enroll(s2))   # יש דרישת קדם ולכן אמור לעבוד
print("רישום עדי לקורס C102:", c2.enroll(s1))   # אמור להיכשל כי אין דרישת קדם

# 9. בדיקת מערכת שעות + חפיפות
print("\n--- מערכת שעות וחפיפות ---")
schedule, overlaps = dep.generate_schedule_report()
print("מערכת שעות:", schedule)
print("חפיפות:", overlaps)

# -----------------------------
# מערכת ספרייה
# -----------------------------
print("\n--- ספרייה ---")
lib = libary("ספריית הנדסה") # Corrected class name

b1 = Book("B1", "Operations Research", "Hillier", c1) # Added course to Book
b2 = Book("B2", "Data Analytics", "Provost", c2) # Added course to Book

lib.add_book(b1)
lib.add_book(b2)

# חיפוש ספרים
print("חיפוש 'Data':", [b.title for b in lib.search("Data")]) # Search term should be string

#--השאלת ספרים--
print("השאלת Data Analytics:")
b2 = lib.search("Data")[0]   # או למצוא לפי ID
b2.borrow(s2) # Pass the Student object, not a string

print("דו״ח ספרים מושאלים:")
print(lib.borrowed_report())


# החזרה--
b2.return_book()
print("דו״ח אחרי החזרה:")
print(lib.borrowed_report())

# Example of submitting grades and analyzing statistics
# lec1.submit_final_grade(c1)
# print("\n--- Teaching Statistics for", lec1.name, "---")
# print(lec1.analyze_teaching_statistics())

# lec2.submit_final_grade(c2)
# print("\n--- Teaching Statistics for", lec2.name, "---")
# print(lec2.analyze_teaching_statistics())

course מבוא לתעשייה וניהול was added with grade 90.

--- רישום לקורסים ---
רישום עדי לקורס C101: True
רישום אור לקורס C102: True
רישום עדי לקורס C102: False

--- מערכת שעות וחפיפות ---
מערכת שעות: {('Mon', 10, 12): ['מבוא לתעשייה וניהול', 'סטטיסטיקה']}
חפיפות: [(('Mon', 10, 12), ['מבוא לתעשייה וניהול', 'סטטיסטיקה'])]

--- ספרייה ---
חיפוש 'Data': ['Data Analytics']
השאלת Data Analytics:
Or borrowed the book Data Analytics by Provost.
דו״ח ספרים מושאלים:
['Data Analytics']
Or returned the book Data Analytics by Provost.
דו״ח אחרי החזרה:
[]
